In [1]:
!pip install selenium

In [2]:
!apt-get update
!apt-get install chromium-chromedriver

'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [2]:
from selenium import webdriver
import sys
import pandas as pd
from selenium.webdriver.chrome.options import Options
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from selenium.common.exceptions import InvalidSessionIdException


In [3]:

options = webdriver.ChromeOptions()
#options.add_argument('headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('disable-gpu')
options.add_argument('lang=ko_KR')
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
#notification 없애기
prefs = {"profile.default_content_setting_values.notifications": 2}
options.add_experimental_option("prefs", prefs)


driver = webdriver.Chrome('../chromedriver', options=options)


In [4]:
def cleansing(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail주소제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]' # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    return text    


In [5]:

#로그인

LOGIN_URL = 'https://www.instagram.com/'

driver.get(LOGIN_URL)
time.sleep(2)
id = 'the_bettersun'
password = 'Eun_5375353!'
id_input = driver.find_elements_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input')[0]
id_input.send_keys(id)
password_input = driver.find_elements_by_css_selector('#loginForm > div > div:nth-child(2) > div > label > input')[0]
password_input.send_keys(password)
password_input.submit()

time.sleep(4)

driver.find_elements_by_xpath('/html/body/div[1]/section/main/div/div/div/div/button')[0].click()
time.sleep(4)



LOGIN_INFO = {
    'id': 'the_bettersun',
    'pass': 'Eun_5375353!'
}
s = requests.Session()

with requests.Session() as s:
    res = s.post(LOGIN_URL, data=LOGIN_INFO, verify=False, allow_redirects=False)
    print(res.headers.keys)
    # 쿠키와 헤더에 포함된 302 Location 값을 가져온다.
    # 이때, 헤더에 설정된 쿠키와 함께 Location으로 Get Request 를 보내면 된다.
    redirect_cookie = res.headers['Set-Cookie']
    print(redirect_cookie)
    #redirect_url = res.headers['Location']
    #print(redirect_url)
    headers = {"Cookie": redirect_cookie}

    # Location 주소로 Get Request 호출
    s.get(LOGIN_URL, headers=headers)


time.sleep(1)


c:\users\07\anaconda3\envs\machinelearning\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.instagram.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<bound method Mapping.keys of {'Content-Type': 'text/html; charset=utf-8', 'Vary': 'Accept-Language, Cookie', 'Content-Language': 'en', 'Date': 'Tue, 20 Oct 2020 04:14:48 GMT', 'Strict-Transport-Security': 'max-age=31536000', 'Cache-Control': 'private, no-cache, no-store, must-revalidate', 'Pragma': 'no-cache', 'Expires': 'Sat, 01 Jan 2000 00:00:00 GMT', 'X-Frame-Options': 'SAMEORIGIN', 'content-security-policy': "report-uri https://www.instagram.com/security/csp_report/; default-src 'self' https://www.instagram.com; img-src data: blob: https://*.fbcdn.net https://*.instagram.com https://*.cdninstagram.com https://*.facebook.com https://*.giphy.com; font-src data: https://*.fbcdn.net https://*.instagram.com https://*.cdninstagram.com; media-src 'self' blob: https://www.instagram.com https://*.cdninstagram.com https://*.fbcdn.net; manifest-src 'self' https://www.instagram.com; script-src 'self' https://instagram.com https://www.instagram.com https://*.www.instagram.com https://*.cdninst

c:\users\07\anaconda3\envs\machinelearning\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.instagram.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [6]:
#키워드 크롤링 시작
keyword = "치즈볼"

url = "https://www.instagram.com/explore/tags/{}/".format(keyword)
instagram_title = []
instagram_dates = []
instagram_likes = []
instagram_name = []
driver.get(url)
time.sleep(3)
count = driver.find_element_by_css_selector('#react-root > section > main > header > div.WSpok > div > div.Igw0E.IwRSH.eGOV_._4EzTm.a39_R > span > span').text
count = count.replace(',','')
count = int(count)
print(count)

driver.find_element_by_css_selector('div.v1Nh3.kIKUG._bz0w').click()
for i in range(1,count):
    time.sleep(3)
    try:
        
        #게시자 이름 가져오기
        name = driver.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/div[3]/div[1]/ul/div/li/div/div/div[2]/h2/div/span/a').text

        #글내용 가져오기
        title = driver.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span').text
        title = cleansing(title)
        title = title.replace('\n', ' ')


        #날짜 가져오기
        date = driver.find_elements_by_xpath('/html/body/div[4]/div[2]/div/article/div[3]/div[2]/a/time') #날짜가 들어있는 xpath
        date = date[0].get_attribute('title')  #날짜만 가져오기
        
        try:
            # 좋아요수 가져오기, 없을 경우 0으로
            likes = driver.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/div[3]/section[2]/div/div/button/span').text
        except:
            likes = 0
            
        instagram_name.append(name)
        instagram_title.append(title)
        instagram_dates.append(date)
        instagram_likes.append(likes)

    except:
        print(i, 'title/dates/likes error')
        #도중에 에러가 발생했을 경우, 내용과 날짜에 NaN값을 추가

        instagram_title.append("NaN")
        instagram_dates.append('NaN')
        instagram_likes.append('NaN')
        instagram_name.append('NaN')

    try:
        #다음페이지로 넘어가는 '>'표시 클릭
        driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow').click()

    except:
        print(i, 'next page error')
        #driver.close() # date = datum2.text #print(date)

    if i % 100 == 0:
        df_instagram = pd.DataFrame({'date': instagram_dates,'id':instagram_name, 'title': instagram_title, 'likes' : instagram_likes})
        df_instagram.to_csv('../prj_sns_trend_private/crawling_instagram_치즈볼_{}.csv'.format(i))
        print(i)
    time.sleep(1)


153192
100
200
300
400
500
600
700
800
900
972 title/dates/likes error
1000
1100
1182 title/dates/likes error
1200
1300
1400
1500
1600
1700
1709 title/dates/likes error
1710 title/dates/likes error
1800
1900
2000
2100
2200
2274 title/dates/likes error
2300
2400
2485 title/dates/likes error
2500
2600
2650 title/dates/likes error
2653 title/dates/likes error
2700
2746 title/dates/likes error
2800
2900
3000
3007 title/dates/likes error
3100
3122 title/dates/likes error
3158 title/dates/likes error
3181 title/dates/likes error
3200
3300
3334 title/dates/likes error
3400
3500
3600
3700
3800
3900
3973 title/dates/likes error
4000
4100
4200
4276 title/dates/likes error
4300
4339 title/dates/likes error
4400
4500
4600
4700
4800
4900
4986 title/dates/likes error
5000
5100
5168 title/dates/likes error
5200
5277 title/dates/likes error
5300
5400
5500
5600
5700
5800
5900
5964 title/dates/likes error
6000
6063 title/dates/likes error
6077 title/dates/likes error
6100
6200
6300
6400
6458 title/dates

22212 title/dates/likes error
22212 next page error
22213 title/dates/likes error
22213 next page error
22214 title/dates/likes error
22214 next page error
22215 title/dates/likes error
22215 next page error
22216 title/dates/likes error
22216 next page error
22217 title/dates/likes error
22217 next page error
22218 title/dates/likes error
22218 next page error
22219 title/dates/likes error
22219 next page error
22220 title/dates/likes error
22220 next page error
22221 title/dates/likes error
22221 next page error
22222 title/dates/likes error
22222 next page error
22223 title/dates/likes error
22223 next page error
22224 title/dates/likes error
22224 next page error
22225 title/dates/likes error
22225 next page error
22226 title/dates/likes error
22226 next page error
22227 title/dates/likes error
22227 next page error
22228 title/dates/likes error
22228 next page error
22229 title/dates/likes error
22229 next page error
22230 title/dates/likes error
22230 next page error
22231 title/

22370 title/dates/likes error
22370 next page error
22371 title/dates/likes error
22371 next page error
22372 title/dates/likes error
22372 next page error
22373 title/dates/likes error
22373 next page error
22374 title/dates/likes error
22374 next page error
22375 title/dates/likes error
22375 next page error
22376 title/dates/likes error
22376 next page error
22377 title/dates/likes error
22377 next page error
22378 title/dates/likes error
22378 next page error
22379 title/dates/likes error
22379 next page error
22380 title/dates/likes error
22380 next page error
22381 title/dates/likes error
22381 next page error
22382 title/dates/likes error
22382 next page error
22383 title/dates/likes error
22383 next page error
22384 title/dates/likes error
22384 next page error
22385 title/dates/likes error
22385 next page error
22386 title/dates/likes error
22386 next page error
22387 title/dates/likes error
22387 next page error
22388 title/dates/likes error
22388 next page error
22389 title/

22528 title/dates/likes error
22528 next page error
22529 title/dates/likes error
22529 next page error
22530 title/dates/likes error
22530 next page error
22531 title/dates/likes error
22531 next page error
22532 title/dates/likes error
22532 next page error
22533 title/dates/likes error
22533 next page error
22534 title/dates/likes error
22534 next page error
22535 title/dates/likes error
22535 next page error
22536 title/dates/likes error
22536 next page error
22537 title/dates/likes error
22537 next page error
22538 title/dates/likes error
22538 next page error
22539 title/dates/likes error
22539 next page error
22540 title/dates/likes error
22540 next page error
22541 title/dates/likes error
22541 next page error
22542 title/dates/likes error
22542 next page error
22543 title/dates/likes error
22543 next page error
22544 title/dates/likes error
22544 next page error
22545 title/dates/likes error
22545 next page error
22546 title/dates/likes error
22546 next page error
22547 title/

22686 title/dates/likes error
22686 next page error
22687 title/dates/likes error
22687 next page error
22688 title/dates/likes error
22688 next page error
22689 title/dates/likes error
22689 next page error
22690 title/dates/likes error
22690 next page error
22691 title/dates/likes error
22691 next page error
22692 title/dates/likes error
22692 next page error
22693 title/dates/likes error
22693 next page error
22694 title/dates/likes error
22694 next page error
22695 title/dates/likes error
22695 next page error
22696 title/dates/likes error
22696 next page error
22697 title/dates/likes error
22697 next page error
22698 title/dates/likes error
22698 next page error
22699 title/dates/likes error
22699 next page error
22700 title/dates/likes error
22700 next page error
22700
22701 title/dates/likes error
22701 next page error
22702 title/dates/likes error
22702 next page error
22703 title/dates/likes error
22703 next page error
22704 title/dates/likes error
22704 next page error
22705 

22844 title/dates/likes error
22844 next page error
22845 title/dates/likes error
22845 next page error
22846 title/dates/likes error
22846 next page error
22847 title/dates/likes error
22847 next page error
22848 title/dates/likes error
22848 next page error
22849 title/dates/likes error
22849 next page error
22850 title/dates/likes error
22850 next page error
22851 title/dates/likes error
22851 next page error
22852 title/dates/likes error
22852 next page error
22853 title/dates/likes error
22853 next page error
22854 title/dates/likes error
22854 next page error
22855 title/dates/likes error
22855 next page error
22856 title/dates/likes error
22856 next page error
22857 title/dates/likes error
22857 next page error
22858 title/dates/likes error
22858 next page error
22859 title/dates/likes error
22859 next page error
22860 title/dates/likes error
22860 next page error
22861 title/dates/likes error
22861 next page error
22862 title/dates/likes error
22862 next page error
22863 title/

KeyboardInterrupt: 

In [16]:
aa = driver.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/div[3]/section[2]/div/div/button/span').text

print(aa)

172


In [ ]:
df_instagram= pd.DataFrame({'date': instagram_title_dates, 'title' : instagram_title})
df_instagram.to_csv('prj_sns_trend_crawling_instagram_test.csv')

In [ ]:
title = driver.find_element_by_xpath('/html/body/div[4]/div[2]/div/article/div[3]/div[1]/ul/div/li/div/div/div[2]/span').text

title = cleansing(title)
title = title.replace('\n', ' ')
print(title)
a = []
a.append(title)
print(a)

#title = re.findall('[A-Za-z0-9가-힣, +]+', title)
#title = ' '.join(title)

맛스타그램 먹스타그램 쿠캣마켓 모짜통통치즈볼 바삭 말랑 단짠 자제 안 하고 먹으면 1봉지 다 먹을삘 치즈볼 만드신 분 칭찬해
['맛스타그램 먹스타그램 쿠캣마켓 모짜통통치즈볼 바삭 말랑 단짠 자제 안 하고 먹으면 1봉지 다 먹을삘 치즈볼 만드신 분 칭찬해']
